In [1]:
import pandas as pd

df = pd.read_csv("train.csv")
df

,Age,BMI,High Blood Pressure,Fasting Blood Glucose,HbA1c Level,Smoking,Diagnosis
0,63,27,1,158,6.5,0,0
1,25,25,0,85,5.0,0,0
2,21,28,0,155,5.8,0,0
3,47,29,0,159,3.5,1,0
4,53,29,0,126,5.8,1,0
...,...,...,...,...,...,...,...
70699,62,27,0,160,4.0,0,0
70700,35,31,1,140,6.0,1,0
70701,56,31,0,160,5.7,0,1
70702,37,21,0,159,6.5,0,0


In [2]:
x = df[["Age", "BMI", "HbA1c Level", "Fasting Blood Glucose", "High Blood Pressure", "Smoking"]].to_numpy()
y = df["Diagnosis"].to_numpy()

print(x.shape, y.shape)

(70704, 6) (70704,)


In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y,random_state=42)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(53028, 6) (53028,)
(17676, 6) (17676,)


In [7]:
x_sub, x_val, y_sub, y_val = train_test_split(x_train, y_train, random_state=42)

print(x_sub.shape, x_val.shape)

(39771, 6) (13257, 6)


In [8]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(x_sub)

x_sub_scaled = ss.transform(x_sub)
x_val_scaled = ss.transform(x_val)

In [9]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(x_sub_scaled, y_sub)

print("학습용 데이터셋:", lr.score(x_sub_scaled, y_sub))
print("시험용 데이터셋:", lr.score(x_val_scaled, y_val))

학습용 데이터셋: 0.955620929823238
시험용 데이터셋: 0.9558723693143245


In [10]:
ss = StandardScaler()
ss.fit(x_train)

x_train_scaled = ss.transform(x_train)
x_test_scaled = ss.transform(x_test)

In [21]:
from sklearn.model_selection import cross_validate

scores = cross_validate(lr, x_train_scaled, y_train)
print(scores)

{'fit_time': array([0.17453313, 0.07978654, 0.09574151, 0.08975911, 0.08477378]), 'score_time': array([0.00299025, 0.00299191, 0.0039897 , 0.00299191, 0.00399256]), 'test_score': array([0.95549689, 0.95596832, 0.95568546, 0.95473833, 0.95643564])}


In [22]:
import numpy as np
print(np.mean(scores["test_score"]))

0.9556649287945416


In [20]:
# from sklearn.model_selection import StratifiedKFold

# splitter = StratifiedKFold(n_splits=9, shuffle=True, random_state=42)
# scores = cross_validate(lr, x_train_scaled, y_train, cv=splitter)
# print(np.mean(scores["test_score"]))

0.9557215056196726


- 그리드서치

In [23]:
from sklearn.model_selection import GridSearchCV

params = {"penalty": ["l1", "l2"],
          "tol": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
          'C': [1e-3, 1e-2, 1e-1, 1, 2, 3, 4],
          'solver': ["liblinear", "saga"]
          }


gs = GridSearchCV(LogisticRegression(), params, n_jobs=-1)

gs.fit(x_train_scaled, y_train)

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 2, 3, 4],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga'],
                         'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1]})

In [24]:
print(gs.best_params_)

{'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear', 'tol': 0.01}


In [25]:
print(np.max(gs.cv_results_["mean_test_score"]))

0.9558535066351116


In [26]:
lr = gs.best_estimator_
print(lr.score(x_train_scaled, y_train))

0.9557780795051671


- 랜덤 서치

In [27]:
params = {"penalty": ["l1", "l2"],
          "tol": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
          'C': [1e-3, 1e-2, 1e-1, 1, 2, 3, 4],
          'solver': ["liblinear", "saga"]
          }

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(LogisticRegression(solver="liblinear"), params,
                                           n_iter=100, n_jobs=-1, random_state=42)

gs.fit(x_train_scaled, y_train)

RandomizedSearchCV(estimator=LogisticRegression(solver='liblinear'), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'C': [0.001, 0.01, 0.1, 1, 2, 3, 4],
                                        'penalty': ['l1', 'l2'],
                                        'solver': ['liblinear', 'saga'],
                                        'tol': [1e-05, 0.0001, 0.001, 0.01,
                                                0.1]},
                   random_state=42)

In [28]:
print(gs.best_params_)

{'tol': 0.01, 'solver': 'liblinear', 'penalty': 'l2', 'C': 0.1}


In [29]:
print(np.max(gs.cv_results_["mean_test_score"]))

0.9558535066351116


In [30]:
lr = gs.best_estimator_
print(lr.score(x_train_scaled, y_train))

0.9557780795051671
